In [1]:
from vocos import Vocos
import torch
import librosa
import soundfile as sf
import os 
import numpy as np
import torchaudio

def file_pathname(target_dir, target_suffix=".wav"):
    find_res = []
    target_suffix_dot = target_suffix
    walk_generator = os.walk(target_dir)
    for root_path, dirs, files in walk_generator:
        if len(files) < 1:
            continue
        for file in files:
            file_name, suffix_name = os.path.splitext(file)
            if suffix_name == target_suffix_dot:
                find_res.append((os.path.normpath(root_path), file_name))
    return find_res

def check_path(path1):
    if type(path1) == list:
        flag = True
        for p in path1:
            flag_tmp = check_path(p)
            flag = (flag and flag_tmp)
        return flag
    flag = os.path.isdir(path1)
    if not flag:
        os.makedirs(path1)
        print(path1+' has been created')
    return flag


/data3/tansongbin/anaconda/envs/bwe_smac/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from vocos.experiment import VocosExp


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print(device)
vocos = Vocos.from_pretrained("charactr/vocos-mel-24khz").to(device)
# vocos=Vocos.from_hparams('/data3/tansongbin/vocoder_projects/vocos/logs/lightning_logs/version_11/config.yaml')


cuda


_IncompatibleKeys(missing_keys=[], unexpected_keys=['multiperioddisc.discriminators.0.convs.0.bias', 'multiperioddisc.discriminators.0.convs.0.weight_g', 'multiperioddisc.discriminators.0.convs.0.weight_v', 'multiperioddisc.discriminators.0.convs.1.bias', 'multiperioddisc.discriminators.0.convs.1.weight_g', 'multiperioddisc.discriminators.0.convs.1.weight_v', 'multiperioddisc.discriminators.0.convs.2.bias', 'multiperioddisc.discriminators.0.convs.2.weight_g', 'multiperioddisc.discriminators.0.convs.2.weight_v', 'multiperioddisc.discriminators.0.convs.3.bias', 'multiperioddisc.discriminators.0.convs.3.weight_g', 'multiperioddisc.discriminators.0.convs.3.weight_v', 'multiperioddisc.discriminators.0.convs.4.bias', 'multiperioddisc.discriminators.0.convs.4.weight_g', 'multiperioddisc.discriminators.0.convs.4.weight_v', 'multiperioddisc.discriminators.0.conv_post.bias', 'multiperioddisc.discriminators.0.conv_post.weight_g', 'multiperioddisc.discriminators.0.conv_post.weight_v', 'multiperiod

odict_keys(['feature_extractor.mel_spec.spectrogram.window', 'feature_extractor.mel_spec.mel_scale.fb', 'backbone.embed.weight', 'backbone.embed.bias', 'backbone.norm.weight', 'backbone.norm.bias', 'backbone.convnext.0.gamma', 'backbone.convnext.0.dwconv.weight', 'backbone.convnext.0.dwconv.bias', 'backbone.convnext.0.norm.weight', 'backbone.convnext.0.norm.bias', 'backbone.convnext.0.pwconv1.weight', 'backbone.convnext.0.pwconv1.bias', 'backbone.convnext.0.pwconv2.weight', 'backbone.convnext.0.pwconv2.bias', 'backbone.convnext.1.gamma', 'backbone.convnext.1.dwconv.weight', 'backbone.convnext.1.dwconv.bias', 'backbone.convnext.1.norm.weight', 'backbone.convnext.1.norm.bias', 'backbone.convnext.1.pwconv1.weight', 'backbone.convnext.1.pwconv1.bias', 'backbone.convnext.1.pwconv2.weight', 'backbone.convnext.1.pwconv2.bias', 'backbone.convnext.2.gamma', 'backbone.convnext.2.dwconv.weight', 'backbone.convnext.2.dwconv.bias', 'backbone.convnext.2.norm.weight', 'backbone.convnext.2.norm.bias',

In [4]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device='cpu'
vocos = Vocos.from_pretrained("charactr/vocos-mel-24khz").to(device)
ckpt_path = '/data3/tansongbin/vocoder_projects/vocos/logs/lightning_logs/version_13/checkpoints/vocos_checkpoint_epoch=11_step=75720_val_loss=4.7365.ckpt'
interface = torch.load(ckpt_path).to(device)
vocos.load_state_dict(interface['state_dict'],strict=False)

inpath=r'/data2/tansongbin/bwe_dataset/online_data/online_fix/benchmark1/input'
outpath=r'/data2/tansongbin/bwe_dataset/online_data/online_fix/benchmark1/output1'
check_path(outpath)

L = file_pathname(inpath,'.wav')
for p,n in L[:10]:
    y, sr = torchaudio.load(os.path.join(p,n+'.wav'))
    if y.size(0) > 1:  # mix to mono
        y = y.mean(dim=0, keepdim=True)
    y = torchaudio.functional.resample(y, orig_freq=sr, new_freq=24000).to(device)
    y_g_hat = vocos(y)
    audio = y_g_hat.squeeze().cpu().numpy()
    audio = librosa.resample(y=audio, orig_sr=24000, target_sr=16000)
    if np.max(np.abs(audio)) > 1.0:
        print(np.max(np.abs(audio)))
        audio/=np.max(np.abs(audio))
    output_file = os.path.join(outpath, n+'_generated.wav')
    sf.write(output_file, audio, 16000)
    print(output_file)

1.696018
/data2/tansongbin/bwe_dataset/online_data/online_fix/benchmark1/output1/downsampled_online_generated.wav


In [4]:
# sf.write(output_file, audio, 16000)


array([ 3.6177430e-06,  5.3062704e-06, -1.8884533e-06, ...,
       -5.1196505e-05, -2.7590577e-04,  0.0000000e+00], dtype=float32)

NameError: name 'interface' is not defined